In [2]:
import os
import re
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from utils import *

In [3]:
df_meta = pd.read_csv('benchmark_meta_data.csv', sep='\t')
df_db = pd.read_csv('track_2023_database.csv', sep='\t')
df = df_meta.merge(df_db, how='inner', left_on='name', right_on='filename')
df.drop(columns='filename', inplace=True)
df

,name,variables,clauses,hash,isohash,family,author,result,proceedings,minisat1m,track
0,hash_table_find_safety_size_23.cnf,14140522,64659736,5a72cb588bcda7b1b54a1eac6b19ab2e,b99f5140b44b4bc0b8537a774833fa31,hashtable-safety,osama,unsat,http://hdl.handle.net/10138/563824,no,"main_2023,submissions_2023"
1,WCNF_from_fp_13.cnf,572,12199,21bde0ebb111c637a1ae8789827b0b73,0a9f1ed661705f7290b1844529449278,register-allocation,green,unsat,http://hdl.handle.net/10138/563824,no,"main_2023,submissions_2023"
2,intervals753.cnf,2572,61869,52be8740aab896317320cd44a5d2af2f,3f32bca8a45bc6966ca7524325816668,interval-matching,gao,unsat,http://hdl.handle.net/10138/563824,no,"main_2023,submissions_2023"
3,brent_69_0.3.cnf,81990,332066,1932e1e8f0c78372759909e45a19ecc7,bde74a7458d922c1ad218222b0c90548,brent-equations,karia,sat,http://hdl.handle.net/10138/563824,no,"main_2023,submissions_2023"
4,WS_500_32_50_10.apx_2_DC-AD.cnf,1000,25001,275d36e3f246990dccd094e5e1442ba9,1e229b6204e3dda99b911b0d190f6d3a,argumentation,niskanen,unknown,http://hdl.handle.net/10138/563824,no,"main_2023,submissions_2023"
...,...,...,...,...,...,...,...,...,...,...,...
395,grs-32-256.cnf,839258,2383464,dbe18c99e95ac42816b99b96d36a8732,775ce77204302af5d01e623c666327cc,grs-fp-comm,fleury,unsat,http://hdl.handle.net/10138/563824,no,"main_2023,submissions_2023"
396,SCPC-800-50.cnf,800,33848,c1324a6876269ee93890e266fe214557,ee18746bf9596c06767acbed26e343e1,set-covering,zheng,sat,http://hdl.handle.net/10138/563824,no,"main_2023,submissions_2023"
397,hash_table_find_safety_size_12.cnf,5861110,27150271,685d37e93ba26197925353e58026ae10,67aa30d697e528bc08fc4f1d7a2c1c98,hashtable-safety,osama,unsat,http://hdl.handle.net/10138/563824,no,"main_2023,submissions_2023"
398,CNP-5-1100.cnf,22585,167911,ce49f590c98885f7ac95567f2b1e7dd5,618566aa569eb9b1469f8cc7d8030a5b,coloring,heule,sat,https://helda.helsinki.fi/bitstream/handle/101...,no,"anni_2022,main_2018,main_2023"


In [ ]:
subset = df[df['variables'] > (10**6) * 5].sort_values(by='name')
h = {}
for i, row in subset.iterrows():
    family_name = row['family']
    if not h.__contains__(family_name):
        h[family_name] = 0
    h[family_name] += 1

# len(subset['name'])
# sum(h.values())
# list(h)
h

{'profitable-robust-production': 6,
 'hashtable-safety': 19,
 'risc-instruction-removal-subrv': 2}

### 40 Instances with more thannn 10⁶ variables
### distributed across 8 benchmark families:
Family | contained_in_cluster / family_size\
profitable-robust-production: 10/20\
cril-misc: 3/3\
baseball-lineup: 1/1\
grs-fp-comm: 1/17\
hashtable-safety: 20/20\
core-based-generator: 1/1\
risc-instruction-removal-subrv: 2/2\
trigonometric-functions: 2/4


In [5]:
subset = df
#df[df['family'] in list(h)]
subset.sort_values(by='name').drop(columns=['hash', 'isohash', 'proceedings'])

,name,variables,clauses,family,author,result,minisat1m,track
106,10pipe_q0_k.cnf,77639,2082017,hardware-verification,velev,unsat,no,"anni_2022,application_2012,application_2013,ap..."
186,15pipe_q0_k.cnf,277976,10103924,hardware-verification,velev,unsat,empty,"anni_2022,application_2012,application_2014,ma..."
318,16_1.cnf,2688,8896,cryptography-simon,zhang,sat,no,"main_2023,submissions_2023"
117,170223547.cnf,322,1127,hgen,guanfeng,sat,no,"anni_2022,main_2020,main_2023"
276,17_0.cnf,2848,9440,cryptography-simon,zhang,sat,no,"main_2023,submissions_2023"
...,...,...,...,...,...,...,...,...
361,tseitingrid7x160_shuffled-sc2016.cnf,2240,8960,tseitin-formulas,elffers,unsat,no,"anni_2022,main_2019,main_2023"
134,unsat-set-b-fclqcolor-10-07-09.sat05-1282.resh...,205,4549,coloring,sabharwal,unsat,no,"anni_2022,crafted_2007,crafted_2009,crafted_20..."
130,velev-pipe-o-uns-1.0-7.cnf,24415,711050,hardware-verification,velev,unsat,no,"anni_2022,application_2009,application_2011,ap..."
200,vmpc_24.cnf,576,67872,cryptography,grieu,sat,yes,"anni_2022,application_2009,industrial_2005,ind..."
